In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import optuna

pd.set_option('max_columns', 300)

In [2]:
df_train = pd.read_csv('../train.csv')
df_train

,row_id,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,A0T0G9C1,A0T0G10C0,A0T1G0C9,A0T1G1C8,A0T1G2C7,A0T1G3C6,A0T1G4C5,A0T1G5C4,A0T1G6C3,A0T1G7C2,A0T1G8C1,A0T1G9C0,A0T2G0C8,A0T2G1C7,A0T2G2C6,A0T2G3C5,A0T2G4C4,A0T2G5C3,A0T2G6C2,A0T2G7C1,A0T2G8C0,A0T3G0C7,A0T3G1C6,A0T3G2C5,A0T3G3C4,A0T3G4C3,A0T3G5C2,A0T3G6C1,A0T3G7C0,A0T4G0C6,A0T4G1C5,A0T4G2C4,A0T4G3C3,A0T4G4C2,A0T4G5C1,A0T4G6C0,A0T5G0C5,A0T5G1C4,A0T5G2C3,A0T5G3C2,A0T5G4C1,A0T5G5C0,A0T6G0C4,A0T6G1C3,A0T6G2C2,A0T6G3C1,A0T6G4C0,A0T7G0C3,A0T7G1C2,A0T7G2C1,A0T7G3C0,A0T8G0C2,A0T8G1C1,A0T8G2C0,A0T9G0C1,A0T9G1C0,A0T10G0C0,A1T0G0C9,A1T0G1C8,A1T0G2C7,A1T0G3C6,A1T0G4C5,A1T0G5C4,A1T0G6C3,A1T0G7C2,A1T0G8C1,A1T0G9C0,A1T1G0C8,A1T1G1C7,A1T1G2C6,A1T1G3C5,A1T1G4C4,A1T1G5C3,A1T1G6C2,A1T1G7C1,A1T1G8C0,A1T2G0C7,A1T2G1C6,A1T2G2C5,A1T2G3C4,A1T2G4C3,A1T2G5C2,A1T2G6C1,A1T2G7C0,A1T3G0C6,A1T3G1C5,A1T3G2C4,A1T3G3C3,A1T3G4C2,A1T3G5C1,A1T3G6C0,A1T4G0C5,A1T4G1C4,A1T4G2C3,A1T4G3C2,A1T4G4C1,A1T4G5C0,A1T5G0C4,A1T5G1C3,A1T5G2C2,A1T5G3C1,A1T5G4C0,A1T6G0C3,A1T6G1C2,A1T6G2C1,A1T6G3C0,A1T7G0C2,A1T7G1C1,A1T7G2C0,A1T8G0C1,A1T8G1C0,A1T9G0C0,A2T0G0C8,A2T0G1C7,A2T0G2C6,A2T0G3C5,A2T0G4C4,A2T0G5C3,A2T0G6C2,A2T0G7C1,A2T0G8C0,A2T1G0C7,A2T1G1C6,A2T1G2C5,A2T1G3C4,A2T1G4C3,A2T1G5C2,A2T1G6C1,A2T1G7C0,A2T2G0C6,A2T2G1C5,A2T2G2C4,A2T2G3C3,A2T2G4C2,A2T2G5C1,A2T2G6C0,A2T3G0C5,A2T3G1C4,A2T3G2C3,A2T3G3C2,A2T3G4C1,A2T3G5C0,A2T4G0C4,A2T4G1C3,A2T4G2C2,A2T4G3C1,A2T4G4C0,A2T5G0C3,A2T5G1C2,A2T5G2C1,A2T5G3C0,A2T6G0C2,A2T6G1C1,A2T6G2C0,A2T7G0C1,A2T7G1C0,A2T8G0C0,A3T0G0C7,A3T0G1C6,A3T0G2C5,A3T0G3C4,A3T0G4C3,A3T0G5C2,A3T0G6C1,A3T0G7C0,A3T1G0C6,A3T1G1C5,A3T1G2C4,A3T1G3C3,A3T1G4C2,A3T1G5C1,A3T1G6C0,A3T2G0C5,A3T2G1C4,A3T2G2C3,A3T2G3C2,A3T2G4C1,A3T2G5C0,A3T3G0C4,A3T3G1C3,A3T3G2C2,A3T3G3C1,A3T3G4C0,A3T4G0C3,A3T4G1C2,A3T4G2C1,A3T4G3C0,A3T5G0C2,A3T5G1C1,A3T5G2C0,A3T6G0C1,A3T6G1C0,A3T7G0C0,A4T0G0C6,A4T0G1C5,A4T0G2C4,A4T0G3C3,A4T0G4C2,A4T0G5C1,A4T0G6C0,A4T1G0C5,A4T1G1C4,A4T1G2C3,A4T1G3C2,A4T1G4C1,A4T1G5C0,A4T2G0C4,A4T2G1C3,A4T2G2C2,A4T2G3C1,A4T2G4C0,A4T3G0C3,A4T3G1C2,A4T3G2C1,A4T3G3C0,A4T4G0C2,A4T4G1C1,A4T4G2C0,A4T5G0C1,A4T5G1C0,A4T6G0C0,A5T0G0C5,A5T0G1C4,A5T0G2C3,A5T0G3C2,A5T0G4C1,A5T0G5C0,A5T1G0C4,A5T1G1C3,A5T1G2C2,A5T1G3C1,A5T1G4C0,A5T2G0C3,A5T2G1C2,A5T2G2C1,A5T2G3C0,A5T3G0C2,A5T3G1C1,A5T3G2C0,A5T4G0C1,A5T4G1C0,A5T5G0C0,A6T0G0C4,A6T0G1C3,A6T0G2C2,A6T0G3C1,A6T0G4C0,A6T1G0C3,A6T1G1C2,A6T1G2C1,A6T1G3C0,A6T2G0C2,A6T2G1C1,A6T2G2C0,A6T3G0C1,A6T3G1C0,A6T4G0C0,A7T0G0C3,A7T0G1C2,A7T0G2C1,A7T0G3C0,A7T1G0C2,A7T1G1C1,A7T1G2C0,A7T2G0C1,A7T2G1C0,A7T3G0C0,A8T0G0C2,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0,target
0,0,-9.536743e-07,-9.536743e-06,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,-9.536743e-06,-9.536743e-07,-9.536743e-06,-8.583069e-05,-0.000343,-0.000801,-0.001202,-0.001202,-0.000801,-0.000343,-0.000086,-0.000010,-4.291534e-05,-0.000343,-0.001202,-0.002403,-0.003004,-0.002403,-0.001202,-0.000343,-0.000043,-0.000114,-0.000801,-0.002403,-0.004005,-0.004005,-0.002403,-0.000801,-0.000114,-0.000200,-0.001202,-0.003004,-0.004005,-0.003004,-0.001202,-0.000200,-0.000240,-0.001202,-0.002403,-0.002403,-0.001202,-0.000240,-0.000200,-0.000801,-0.001202,-0.000801,-0.000200,-0.000114,-0.000343,-0.000343,-0.000114,-0.000043,-0.000086,-0.000043,-0.000010,-0.000010,-9.536743e-07,-9.536743e-06,-0.000086,-0.000343,-0.000801,-0.001202,-0.001202,-0.000801,-0.000343,-0.000086,-0.000010,-0.000086,-0.000687,-0.002403,-0.004807,-0.006008,-0.004807,-0.002403,-0.000687,-0.000086,-0.000343,-0.002403,-0.007210,-0.002016,-0.012016,-0.007210,0.017597,0.009657,-0.000801,-0.004807,0.007984,-0.006022,0.007984,-0.004807,-0.000801,-0.001202,0.003992,-0.012016,-0.012016,0.003992,0.008798,-0.001202,-0.004807,0.002790,-0.004807,-0.001202,-0.000801,0.007597,0.007597,-0.000801,-0.000343,-0.000687,-0.000343,-0.000086,-0.000086,-0.000010,-0.000043,-0.000343,-0.001202,-0.002403,-0.003004,-0.002403,-0.001202,-0.000343,-0.000043,-0.000343,-0.002403,-0.007210,-0.012016,-0.002016,-0.007210,-0.002403,-0.0003

In [3]:
df_test = pd.read_csv('../test.csv')
df_test

,row_id,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,A0T0G9C1,A0T0G10C0,A0T1G0C9,A0T1G1C8,A0T1G2C7,A0T1G3C6,A0T1G4C5,A0T1G5C4,A0T1G6C3,A0T1G7C2,A0T1G8C1,A0T1G9C0,A0T2G0C8,A0T2G1C7,A0T2G2C6,A0T2G3C5,A0T2G4C4,A0T2G5C3,A0T2G6C2,A0T2G7C1,A0T2G8C0,A0T3G0C7,A0T3G1C6,A0T3G2C5,A0T3G3C4,A0T3G4C3,A0T3G5C2,A0T3G6C1,A0T3G7C0,A0T4G0C6,A0T4G1C5,A0T4G2C4,A0T4G3C3,A0T4G4C2,A0T4G5C1,A0T4G6C0,A0T5G0C5,A0T5G1C4,A0T5G2C3,A0T5G3C2,A0T5G4C1,A0T5G5C0,A0T6G0C4,A0T6G1C3,A0T6G2C2,A0T6G3C1,A0T6G4C0,A0T7G0C3,A0T7G1C2,A0T7G2C1,A0T7G3C0,A0T8G0C2,A0T8G1C1,A0T8G2C0,A0T9G0C1,A0T9G1C0,A0T10G0C0,A1T0G0C9,A1T0G1C8,A1T0G2C7,A1T0G3C6,A1T0G4C5,A1T0G5C4,A1T0G6C3,A1T0G7C2,A1T0G8C1,A1T0G9C0,A1T1G0C8,A1T1G1C7,A1T1G2C6,A1T1G3C5,A1T1G4C4,A1T1G5C3,A1T1G6C2,A1T1G7C1,A1T1G8C0,A1T2G0C7,A1T2G1C6,A1T2G2C5,A1T2G3C4,A1T2G4C3,A1T2G5C2,A1T2G6C1,A1T2G7C0,A1T3G0C6,A1T3G1C5,A1T3G2C4,A1T3G3C3,A1T3G4C2,A1T3G5C1,A1T3G6C0,A1T4G0C5,A1T4G1C4,A1T4G2C3,A1T4G3C2,A1T4G4C1,A1T4G5C0,A1T5G0C4,A1T5G1C3,A1T5G2C2,A1T5G3C1,A1T5G4C0,A1T6G0C3,A1T6G1C2,A1T6G2C1,A1T6G3C0,A1T7G0C2,A1T7G1C1,A1T7G2C0,A1T8G0C1,A1T8G1C0,A1T9G0C0,A2T0G0C8,A2T0G1C7,A2T0G2C6,A2T0G3C5,A2T0G4C4,A2T0G5C3,A2T0G6C2,A2T0G7C1,A2T0G8C0,A2T1G0C7,A2T1G1C6,A2T1G2C5,A2T1G3C4,A2T1G4C3,A2T1G5C2,A2T1G6C1,A2T1G7C0,A2T2G0C6,A2T2G1C5,A2T2G2C4,A2T2G3C3,A2T2G4C2,A2T2G5C1,A2T2G6C0,A2T3G0C5,A2T3G1C4,A2T3G2C3,A2T3G3C2,A2T3G4C1,A2T3G5C0,A2T4G0C4,A2T4G1C3,A2T4G2C2,A2T4G3C1,A2T4G4C0,A2T5G0C3,A2T5G1C2,A2T5G2C1,A2T5G3C0,A2T6G0C2,A2T6G1C1,A2T6G2C0,A2T7G0C1,A2T7G1C0,A2T8G0C0,A3T0G0C7,A3T0G1C6,A3T0G2C5,A3T0G3C4,A3T0G4C3,A3T0G5C2,A3T0G6C1,A3T0G7C0,A3T1G0C6,A3T1G1C5,A3T1G2C4,A3T1G3C3,A3T1G4C2,A3T1G5C1,A3T1G6C0,A3T2G0C5,A3T2G1C4,A3T2G2C3,A3T2G3C2,A3T2G4C1,A3T2G5C0,A3T3G0C4,A3T3G1C3,A3T3G2C2,A3T3G3C1,A3T3G4C0,A3T4G0C3,A3T4G1C2,A3T4G2C1,A3T4G3C0,A3T5G0C2,A3T5G1C1,A3T5G2C0,A3T6G0C1,A3T6G1C0,A3T7G0C0,A4T0G0C6,A4T0G1C5,A4T0G2C4,A4T0G3C3,A4T0G4C2,A4T0G5C1,A4T0G6C0,A4T1G0C5,A4T1G1C4,A4T1G2C3,A4T1G3C2,A4T1G4C1,A4T1G5C0,A4T2G0C4,A4T2G1C3,A4T2G2C2,A4T2G3C1,A4T2G4C0,A4T3G0C3,A4T3G1C2,A4T3G2C1,A4T3G3C0,A4T4G0C2,A4T4G1C1,A4T4G2C0,A4T5G0C1,A4T5G1C0,A4T6G0C0,A5T0G0C5,A5T0G1C4,A5T0G2C3,A5T0G3C2,A5T0G4C1,A5T0G5C0,A5T1G0C4,A5T1G1C3,A5T1G2C2,A5T1G3C1,A5T1G4C0,A5T2G0C3,A5T2G1C2,A5T2G2C1,A5T2G3C0,A5T3G0C2,A5T3G1C1,A5T3G2C0,A5T4G0C1,A5T4G1C0,A5T5G0C0,A6T0G0C4,A6T0G1C3,A6T0G2C2,A6T0G3C1,A6T0G4C0,A6T1G0C3,A6T1G1C2,A6T1G2C1,A6T1G3C0,A6T2G0C2,A6T2G1C1,A6T2G2C0,A6T3G0C1,A6T3G1C0,A6T4G0C0,A7T0G0C3,A7T0G1C2,A7T0G2C1,A7T0G3C0,A7T1G0C2,A7T1G1C1,A7T1G2C0,A7T2G0C1,A7T2G1C0,A7T3G0C0,A8T0G0C2,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0
0,200000,-9.536743e-07,-0.000002,-9.153442e-07,0.000024,0.000034,-0.000002,0.000021,0.000024,-0.000009,-0.000008,-9.536743e-07,-0.000003,-0.000017,-0.000074,0.000038,0.000068,0.000164,0.000180,-0.000003,-0.000025,-0.000009,0.000004,-0.000101,-0.000171,0.000006,0.000330,0.000580,0.000228,-0.000015,-0.000016,-0.000026,-0.000146,-0.000075,0.000212,0.000565,0.000359,0.000072,-0.000005,-0.000043,-0.000110,0.000061,0.000349,0.000338,0.000163,-0.000003,0.000005,0.000050,0.000332,0.000447,0.000212,0.000016,0.000031,0.000196,0.000331,0.000261,0.000035,0.000064,0.000202,0.000175,0.000047,0.000031,0.000104,0.000026,0.000012,0.000009,-9.536743e-07,4.632568e-07,-0.000016,-0.000005,0.000141,0.000170,0.000091,0.000076,-0.000062,-0.000026,-0.000005,-0.000019,-0.000179,-0.000068,0.000362,0.000505,0.000449,-0.000047,-0.000148,-0.000027,-0.000092,-0.000457,-0.000095,0.000379,0.000940,0.000355,-0.000236,-0.000087,-0.000180,-0.000496,-0.000106,0.000351,0.000539,-0.000045,-0.000107,-0.000103,-0.000306,0.000241,0.000404,0.000074,-0.000065,0.000071,0.000282,0.000549,0.000418,0.000001,0.000214,0.000584,0.000592,0.000200,0.000193,0.000354,0.000146,0.000102,0.000091,0.000024,-0.000011,-0.000030,0.000263,0.000496,0.000333,0.000038,-0.000180,-0.000093,-0.000009,-0.000086,-0.000142,0.000172,0.000703,0.000237,-0.000323,-0.000412,-0.000083,-0.000231,-0.000618,-0.000275,-0.000510,-0.000272,-0.000569,-0.000264,-0.000420,-0.0

In [4]:
df_sample_submission = pd.read_csv('../sample_submission.csv')
df_sample_submission

,row_id,target
0,200000,Streptococcus_pneumoniae
1,200001,Streptococcus_pneumoniae
2,200002,Streptococcus_pneumoniae
3,200003,Streptococcus_pneumoniae
4,200004,Streptococcus_pneumoniae
...,...,...
99995,299995,Streptococcus_pneumoniae
99996,299996,Streptococcus_pneumoniae
99997,299997,Streptococcus_pneumoniae
99998,299998,Streptococcus_pneumoniae


In [6]:
df_train['target'].value_counts()

Bacteroides_fragilis        16111
Streptococcus_pyogenes      16061
Streptococcus_pneumoniae    16059
Campylobacter_jejuni        16050
Salmonella_enterica         16024
Escherichia_coli            15966
Enterococcus_hirae          15958
Escherichia_fergusonii      15950
Staphylococcus_aureus       15943
Klebsiella_pneumoniae       15878
Name: target, dtype: int64

In [7]:
target_values, target_keys = pd.Series(df_train['target'].unique()).factorize()
key2val = dict(zip(target_keys, target_values))
val2key = dict(zip(target_values, target_keys))
df_train['target'] = df_train['target'].replace(key2val)
df_train

,row_id,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,A0T0G9C1,A0T0G10C0,A0T1G0C9,A0T1G1C8,A0T1G2C7,A0T1G3C6,A0T1G4C5,A0T1G5C4,A0T1G6C3,A0T1G7C2,A0T1G8C1,A0T1G9C0,A0T2G0C8,A0T2G1C7,A0T2G2C6,A0T2G3C5,A0T2G4C4,A0T2G5C3,A0T2G6C2,A0T2G7C1,A0T2G8C0,A0T3G0C7,A0T3G1C6,A0T3G2C5,A0T3G3C4,A0T3G4C3,A0T3G5C2,A0T3G6C1,A0T3G7C0,A0T4G0C6,A0T4G1C5,A0T4G2C4,A0T4G3C3,A0T4G4C2,A0T4G5C1,A0T4G6C0,A0T5G0C5,A0T5G1C4,A0T5G2C3,A0T5G3C2,A0T5G4C1,A0T5G5C0,A0T6G0C4,A0T6G1C3,A0T6G2C2,A0T6G3C1,A0T6G4C0,A0T7G0C3,A0T7G1C2,A0T7G2C1,A0T7G3C0,A0T8G0C2,A0T8G1C1,A0T8G2C0,A0T9G0C1,A0T9G1C0,A0T10G0C0,A1T0G0C9,A1T0G1C8,A1T0G2C7,A1T0G3C6,A1T0G4C5,A1T0G5C4,A1T0G6C3,A1T0G7C2,A1T0G8C1,A1T0G9C0,A1T1G0C8,A1T1G1C7,A1T1G2C6,A1T1G3C5,A1T1G4C4,A1T1G5C3,A1T1G6C2,A1T1G7C1,A1T1G8C0,A1T2G0C7,A1T2G1C6,A1T2G2C5,A1T2G3C4,A1T2G4C3,A1T2G5C2,A1T2G6C1,A1T2G7C0,A1T3G0C6,A1T3G1C5,A1T3G2C4,A1T3G3C3,A1T3G4C2,A1T3G5C1,A1T3G6C0,A1T4G0C5,A1T4G1C4,A1T4G2C3,A1T4G3C2,A1T4G4C1,A1T4G5C0,A1T5G0C4,A1T5G1C3,A1T5G2C2,A1T5G3C1,A1T5G4C0,A1T6G0C3,A1T6G1C2,A1T6G2C1,A1T6G3C0,A1T7G0C2,A1T7G1C1,A1T7G2C0,A1T8G0C1,A1T8G1C0,A1T9G0C0,A2T0G0C8,A2T0G1C7,A2T0G2C6,A2T0G3C5,A2T0G4C4,A2T0G5C3,A2T0G6C2,A2T0G7C1,A2T0G8C0,A2T1G0C7,A2T1G1C6,A2T1G2C5,A2T1G3C4,A2T1G4C3,A2T1G5C2,A2T1G6C1,A2T1G7C0,A2T2G0C6,A2T2G1C5,A2T2G2C4,A2T2G3C3,A2T2G4C2,A2T2G5C1,A2T2G6C0,A2T3G0C5,A2T3G1C4,A2T3G2C3,A2T3G3C2,A2T3G4C1,A2T3G5C0,A2T4G0C4,A2T4G1C3,A2T4G2C2,A2T4G3C1,A2T4G4C0,A2T5G0C3,A2T5G1C2,A2T5G2C1,A2T5G3C0,A2T6G0C2,A2T6G1C1,A2T6G2C0,A2T7G0C1,A2T7G1C0,A2T8G0C0,A3T0G0C7,A3T0G1C6,A3T0G2C5,A3T0G3C4,A3T0G4C3,A3T0G5C2,A3T0G6C1,A3T0G7C0,A3T1G0C6,A3T1G1C5,A3T1G2C4,A3T1G3C3,A3T1G4C2,A3T1G5C1,A3T1G6C0,A3T2G0C5,A3T2G1C4,A3T2G2C3,A3T2G3C2,A3T2G4C1,A3T2G5C0,A3T3G0C4,A3T3G1C3,A3T3G2C2,A3T3G3C1,A3T3G4C0,A3T4G0C3,A3T4G1C2,A3T4G2C1,A3T4G3C0,A3T5G0C2,A3T5G1C1,A3T5G2C0,A3T6G0C1,A3T6G1C0,A3T7G0C0,A4T0G0C6,A4T0G1C5,A4T0G2C4,A4T0G3C3,A4T0G4C2,A4T0G5C1,A4T0G6C0,A4T1G0C5,A4T1G1C4,A4T1G2C3,A4T1G3C2,A4T1G4C1,A4T1G5C0,A4T2G0C4,A4T2G1C3,A4T2G2C2,A4T2G3C1,A4T2G4C0,A4T3G0C3,A4T3G1C2,A4T3G2C1,A4T3G3C0,A4T4G0C2,A4T4G1C1,A4T4G2C0,A4T5G0C1,A4T5G1C0,A4T6G0C0,A5T0G0C5,A5T0G1C4,A5T0G2C3,A5T0G3C2,A5T0G4C1,A5T0G5C0,A5T1G0C4,A5T1G1C3,A5T1G2C2,A5T1G3C1,A5T1G4C0,A5T2G0C3,A5T2G1C2,A5T2G2C1,A5T2G3C0,A5T3G0C2,A5T3G1C1,A5T3G2C0,A5T4G0C1,A5T4G1C0,A5T5G0C0,A6T0G0C4,A6T0G1C3,A6T0G2C2,A6T0G3C1,A6T0G4C0,A6T1G0C3,A6T1G1C2,A6T1G2C1,A6T1G3C0,A6T2G0C2,A6T2G1C1,A6T2G2C0,A6T3G0C1,A6T3G1C0,A6T4G0C0,A7T0G0C3,A7T0G1C2,A7T0G2C1,A7T0G3C0,A7T1G0C2,A7T1G1C1,A7T1G2C0,A7T2G0C1,A7T2G1C0,A7T3G0C0,A8T0G0C2,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0,target
0,35656,-9.536743e-07,-9.536743e-06,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,-9.536743e-06,-9.536743e-07,-0.00001,-0.000086,-0.000343,-0.000801,-0.001202,-0.001202,-0.000801,-0.000343,-0.000086,-9.536743e-06,-0.000043,-0.000343,-0.001202,-0.002403,-0.003004,-0.002403,-0.001202,-0.000343,-0.000043,-0.000114,-0.000801,-0.002403,-0.004005,-0.004005,-0.002403,-0.000801,-0.000114,-0.000200,-0.001202,-0.003004,-0.004005,-0.003004,-0.001202,-0.000200,-0.000240,-0.001202,-0.002403,0.007597,-0.001202,-2.403259e-04,-0.00020,-0.000801,-0.001202,-0.000801,-2.002716e-04,-0.000114,-0.000343,-0.000343,-0.000114,-0.000043,-0.000086,-0.000043,-9.536743e-06,-0.000010,-9.536743e-07,-9.536743e-06,-0.000086,-0.000343,-0.000801,-0.001202,-0.001202,-0.000801,-0.000343,-0.000086,-9.536743e-06,-0.000086,-0.000687,-0.002403,-0.004807,-0.006008,-0.004807,-0.002403,-0.000687,-0.000086,-0.000343,-0.002403,-0.007210,-0.012016,-0.002016,-0.007210,-0.002403,-0.000343,-0.000801,-0.004807,-0.012016,-0.006022,-0.012016,-0.004807,-0.000801,-0.001202,-0.006008,-0.002016,-0.012016,0.003992,-0.001202,-0.001202,-0.004807,0.002790,0.005193,-0.001202,-0.000801,-0.002403,-0.002403,0.019199,0.019657,-0.000687,0.009657,-0.000086,-0.000086,-0.000010,-0.000043,-0.000343,-0.001202,-0.002403,-0.003004,-0.002403,-0.001202,-0.000343,-0.000043,-0.000343,-0.002403,-0.00721,-0.002016,-0.012016,-0.007210,-0.00

In [8]:
df_train.corr()

,row_id,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,A0T0G9C1,A0T0G10C0,A0T1G0C9,A0T1G1C8,A0T1G2C7,A0T1G3C6,A0T1G4C5,A0T1G5C4,A0T1G6C3,A0T1G7C2,A0T1G8C1,A0T1G9C0,A0T2G0C8,A0T2G1C7,A0T2G2C6,A0T2G3C5,A0T2G4C4,A0T2G5C3,A0T2G6C2,A0T2G7C1,A0T2G8C0,A0T3G0C7,A0T3G1C6,A0T3G2C5,A0T3G3C4,A0T3G4C3,A0T3G5C2,A0T3G6C1,A0T3G7C0,A0T4G0C6,A0T4G1C5,A0T4G2C4,A0T4G3C3,A0T4G4C2,A0T4G5C1,A0T4G6C0,A0T5G0C5,A0T5G1C4,A0T5G2C3,A0T5G3C2,A0T5G4C1,A0T5G5C0,A0T6G0C4,A0T6G1C3,A0T6G2C2,A0T6G3C1,A0T6G4C0,A0T7G0C3,A0T7G1C2,A0T7G2C1,A0T7G3C0,A0T8G0C2,A0T8G1C1,A0T8G2C0,A0T9G0C1,A0T9G1C0,A0T10G0C0,A1T0G0C9,A1T0G1C8,A1T0G2C7,A1T0G3C6,A1T0G4C5,A1T0G5C4,A1T0G6C3,A1T0G7C2,A1T0G8C1,A1T0G9C0,A1T1G0C8,A1T1G1C7,A1T1G2C6,A1T1G3C5,A1T1G4C4,A1T1G5C3,A1T1G6C2,A1T1G7C1,A1T1G8C0,A1T2G0C7,A1T2G1C6,A1T2G2C5,A1T2G3C4,A1T2G4C3,A1T2G5C2,A1T2G6C1,A1T2G7C0,A1T3G0C6,A1T3G1C5,A1T3G2C4,A1T3G3C3,A1T3G4C2,A1T3G5C1,A1T3G6C0,A1T4G0C5,A1T4G1C4,A1T4G2C3,A1T4G3C2,A1T4G4C1,A1T4G5C0,A1T5G0C4,A1T5G1C3,A1T5G2C2,A1T5G3C1,A1T5G4C0,A1T6G0C3,A1T6G1C2,A1T6G2C1,A1T6G3C0,A1T7G0C2,A1T7G1C1,A1T7G2C0,A1T8G0C1,A1T8G1C0,A1T9G0C0,A2T0G0C8,A2T0G1C7,A2T0G2C6,A2T0G3C5,A2T0G4C4,A2T0G5C3,A2T0G6C2,A2T0G7C1,A2T0G8C0,A2T1G0C7,A2T1G1C6,A2T1G2C5,A2T1G3C4,A2T1G4C3,A2T1G5C2,A2T1G6C1,A2T1G7C0,A2T2G0C6,A2T2G1C5,A2T2G2C4,A2T2G3C3,A2T2G4C2,A2T2G5C1,A2T2G6C0,A2T3G0C5,A2T3G1C4,A2T3G2C3,A2T3G3C2,A2T3G4C1,A2T3G5C0,A2T4G0C4,A2T4G1C3,A2T4G2C2,A2T4G3C1,A2T4G4C0,A2T5G0C3,A2T5G1C2,A2T5G2C1,A2T5G3C0,A2T6G0C2,A2T6G1C1,A2T6G2C0,A2T7G0C1,A2T7G1C0,A2T8G0C0,A3T0G0C7,A3T0G1C6,A3T0G2C5,A3T0G3C4,A3T0G4C3,A3T0G5C2,A3T0G6C1,A3T0G7C0,A3T1G0C6,A3T1G1C5,A3T1G2C4,A3T1G3C3,A3T1G4C2,A3T1G5C1,A3T1G6C0,A3T2G0C5,A3T2G1C4,A3T2G2C3,A3T2G3C2,A3T2G4C1,A3T2G5C0,A3T3G0C4,A3T3G1C3,A3T3G2C2,A3T3G3C1,A3T3G4C0,A3T4G0C3,A3T4G1C2,A3T4G2C1,A3T4G3C0,A3T5G0C2,A3T5G1C1,A3T5G2C0,A3T6G0C1,A3T6G1C0,A3T7G0C0,A4T0G0C6,A4T0G1C5,A4T0G2C4,A4T0G3C3,A4T0G4C2,A4T0G5C1,A4T0G6C0,A4T1G0C5,A4T1G1C4,A4T1G2C3,A4T1G3C2,A4T1G4C1,A4T1G5C0,A4T2G0C4,A4T2G1C3,A4T2G2C2,A4T2G3C1,A4T2G4C0,A4T3G0C3,A4T3G1C2,A4T3G2C1,A4T3G3C0,A4T4G0C2,A4T4G1C1,A4T4G2C0,A4T5G0C1,A4T5G1C0,A4T6G0C0,A5T0G0C5,A5T0G1C4,A5T0G2C3,A5T0G3C2,A5T0G4C1,A5T0G5C0,A5T1G0C4,A5T1G1C3,A5T1G2C2,A5T1G3C1,A5T1G4C0,A5T2G0C3,A5T2G1C2,A5T2G2C1,A5T2G3C0,A5T3G0C2,A5T3G1C1,A5T3G2C0,A5T4G0C1,A5T4G1C0,A5T5G0C0,A6T0G0C4,A6T0G1C3,A6T0G2C2,A6T0G3C1,A6T0G4C0,A6T1G0C3,A6T1G1C2,A6T1G2C1,A6T1G3C0,A6T2G0C2,A6T2G1C1,A6T2G2C0,A6T3G0C1,A6T3G1C0,A6T4G0C0,A7T0G0C3,A7T0G1C2,A7T0G2C1,A7T0G3C0,A7T1G0C2,A7T1G1C1,A7T1G2C0,A7T2G0C1,A7T2G1C0,A7T3G0C0,A8T0G0C2,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0,target
row_id,1.000000,-0.000173,0.000274,0.001161,0.002852,0.000116,0.003881,-0.000201,0.000030,-0.001106,-0.000500,0.001014,-0.002042,0.001636,-0.002702,0.002793,0.001759,0.000452,-0.003510,-0.001838,0.002599,0.003055,0.003387,0.002041,0.001658,-0.001537,0.001633,0.001994,0.000145,-0.001452,0.001329,0.001318,-0.000171,-0.001561,0.001401,0.003069,0.000517,-0.001313,0.003038,-0.002858,-0.003795,0.003574,0.000020,0.002615,0.001378,-0.002582,-0.002636,0.001473,0.002162,-0.001122,-0.001991,-0.001497,-0.002195,-0.000485,0.001928,-0.000701,0.002456,-0.001201,0.002630,-0.001856,0.004030,0.000240,-0.001175,0.002157,0.000839,-0.002470,0.002393,0.002553,0.000111,0.000005,0.000521,0.000795,-0.000925,0.003626,-0.001428,0.001942,0.003957,0.002916,0.003506,0.002252,0.002153,0.000727,0.000766,0.000875,-0.002949,0.002724,0.001535,0.004013,0.002291,0.004142,0.001108,0.002132,0.003971,0.004024,0.004705,0.001555,-0.000143,0.003536,0.003633,0.001553,-0.002328,0.001908,0.004338,0.003942,0.003539,0.003757,0.002369,-0.001131,-0.001267,0.004247,-0.001660,0.002217,-0.000042,0.002921,0.002918,-0.000107,-0.002482,0.003694,0.003357,-0.003027,-0.001504,-0.000644,0.003243,-0.002148,0.001774,0.000517,0.000004,0.002695,0.002231,0.001829,0.003157,0.002485,0.002131,0.001930,0.001046,0.002974,0.001709,0.001408,-0.000273,0.000257,-0.001685,0.002481,0.003231,0.002346,0.001266,0.001267,-0.001097,-0.000212,0.001852,0.001732,0.00

In [9]:
class LGBModel:
    def __init__(self):
        self.models = None
        self.num_folds = None
        self.is_prob = False
        self.num_classes = None

    def fit(self, x, y, num_folds=5, is_plot=False, is_prob=False, seed=0):
        self.num_classes = len(np.unique(y))
        self.is_prob = is_prob
        if is_prob:
            preds = np.zeros((len(x), self.num_classes))
        else:
            preds = np.zeros(len(x))
        self.models = []
        self.num_folds = num_folds
        
        kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=seed)
        for fold, (trn_ind, val_ind) in enumerate(kfold.split(x, y)):
            x_train, y_train = x.loc[trn_ind], y.loc[trn_ind]
            x_val, y_val = x.loc[val_ind], y.loc[val_ind]

            model = lgb.LGBMClassifier()
            model.fit(x_train, y_train)
            if is_prob:
                preds[val_ind] = model.predict_proba(x_val)
            else:
                preds[val_ind] = model.predict(x_val)
                print(f'current_fold: {fold+1}, accuracy: {accuracy_score(y_val, preds[val_ind])}')
            
            self.models.append(model)

        if not is_prob:    
            print(f'Full predictions accuracy: {accuracy_score(y, preds)}')

        if is_plot:
            ax = lgb.plot_importance(self.models[0], max_num_features=50, figsize=(16, 16))
            plt.show()
    
        return preds
    
    def predict(self, x):
        if self.is_prob:
            preds = np.zeros((len(x), self.num_classes))
        else:
            preds = np.zeros(len(x))
        
        for model in self.models:
            if self.is_prob:
                preds += model.predict_proba(x)
            else:
                preds += model.predict(x)
        
        if self.is_prob:
            return preds / self.num_folds
        else:
            return np.round(preds / self.num_folds)

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


seed = 0
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


class NewReprNN(nn.Module):
    def __init__(self, input_size, num_classes, dataset_size, hidden_dim=2):
        super(NewReprNN, self).__init__()
        self.input_size = input_size
        self.hidden_dim = hidden_dim
        self.fc1 = nn.Linear(1, hidden_dim)
        self.fc2 = nn.Linear(input_size*hidden_dim, num_classes)
        self.relu = nn.ReLU()
        self.new_repr = torch.zeros((dataset_size, input_size*hidden_dim))
        self.current_pos = 0
    
    def forward(self, x):
        batch_size = x.shape[0]
        out = self.relu(self.fc1(x.reshape(-1, 1))).reshape(batch_size, -1)
        
        self.new_repr[self.current_pos:self.current_pos+batch_size] = out.detach()
        self.current_pos += batch_size
        
        return self.fc2(out)
    
    def get_new_repr(self):
        return self.new_repr.cpu().numpy()
    
    def reset_new_repr(self, dataset_size):
        self.new_repr = torch.zeros((dataset_size, self.input_size*self.hidden_dim))
        self.current_pos = 0


class NoiseNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NoiseNN, self).__init__()
        self.input_size = input_size
        self.pipe = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_classes)
        )
    
    def forward(self, x):
        return self.pipe(x)
    
    def get_new_repr(self):
        return self.new_repr.cpu().numpy()
    
    def reset_new_repr(self, dataset_size):
        self.new_repr = torch.zeros((dataset_size, self.input_size*self.hidden_dim))
        self.current_pos = 0


class NewReprDataset(Dataset):
    def __init__(self, x, y):
        super(NewReprDataset, self).__init__()
        self.x = x
        self.y = y
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        x = torch.tensor(self.x[index])
        y = torch.tensor(self.y[index])
        
        return x.float(), y


class NewReprModel:
    def __init__(self):
        self.models = None
        self.num_folds = None
        self.device = 'cpu'#'cuda' if torch.cuda.is_available() else 'cpu'
        self.num_classes = None
    
    def fit(self, x, y, num_folds=5, noise_size=10, batch_size=16, lr=3e-4, epochs=1, seed=0):
        self.num_folds = num_folds
        self.models = []
        self.lgb_models = []
        self.noise_models = dict()
        hidden_size_list = []
        hidden_size = 128
        for i in range(noise_size):
            hidden_size_list.append(hidden_size*(i+1))
            self.noise_models[f'model{i+1}'] = []
        self.num_classes = len(np.unique(y.values))
        
        criterion = nn.CrossEntropyLoss()
        
        nn_preds_full = np.zeros((len(x), self.num_classes))
        lgb_preds_full = np.zeros((len(x), self.num_classes))
        noise_preds_full = np.zeros((len(self.noise_models), len(x), self.num_classes))
        
        kfold = StratifiedKFold(n_splits=self.num_folds, shuffle=True, random_state=seed)
        for fold, (trn_ind, val_ind) in enumerate(kfold.split(x, y)):
            x_train, y_train = x.loc[trn_ind].values, y.loc[trn_ind].values
            x_val, y_val = x.loc[val_ind].values, y.loc[val_ind].values
            
            model = NewReprNN(input_size=x_train.shape[1], num_classes=self.num_classes, dataset_size=len(x_train)).to(self.device)
            lgb_model = lgb.LGBMClassifier()
            
            optimizer = optim.Adam(params=model.parameters(), lr=lr)
            
            dataset_train = NewReprDataset(x_train, y_train)
            loader_train = DataLoader(dataset=dataset_train, shuffle=False, batch_size=batch_size)
            
            dataset_val = NewReprDataset(x_val, y_val)
            loader_val = DataLoader(dataset=dataset_val, shuffle=False, batch_size=batch_size)
            
            noise_dataset_train = dict()
            noise_loader_train = dict()
            noise_dataset_val = dict()
            noise_loader_val = dict()
            noise_model = dict()
            for i in range(len(self.noise_models)):
                noise_model[f'model{i+1}'] = NoiseNN(input_size=x_train.shape[1], hidden_size=hidden_size_list[i], num_classes=self.num_classes).to(self.device)
                noise_dataset_train[f'dataset{i+1}'] = NewReprDataset(x_train, y_train)
                noise_loader_train[f'loader{i+1}'] = DataLoader(dataset=noise_dataset_train[f'dataset{i+1}'], shuffle=False, batch_size=batch_size)
                noise_dataset_val[f'dataset{i+1}'] = NewReprDataset(x_val, y_val)
                noise_loader_val[f'loader{i+1}'] = DataLoader(dataset=noise_dataset_val[f'dataset{i+1}'], shuffle=False, batch_size=batch_size)
            
            print(f'\n\ncurrent_fold: {fold+1}')
            
            for epoch in range(epochs):
                
                # Noise Model Train---------------------------------------------
                noise_preds_arr = np.zeros((len(self.noise_models), x_val.shape[0], self.num_classes))
                for i in range(len(self.noise_models)):
                    noise_current_pos = 0
                    noise_model[f'model{i+1}'].train()
                    for data, target in noise_loader_train[f'loader{i+1}']:
                        data = data.to(self.device)
                        target = target.to(self.device)

                        preds = noise_model[f'model{i+1}'](data)
                        loss = criterion(preds, target)

                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                    with torch.no_grad():
                        noise_model[f'model{i+1}'].eval()
                        for data, target in noise_loader_val[f'loader{i+1}']:
                            data = data.to(self.device)
                            target = target.to(self.device)

                            preds = noise_model[f'model{i+1}'](data)
                            noise_preds_arr[i, noise_current_pos:noise_current_pos+preds.shape[0]] = preds.cpu().numpy()
                            noise_current_pos += preds.shape[0]

                        if epoch+1 == epochs:
                            noise_preds_full[i, val_ind] = noise_preds_arr[i]
                # Noise Model Train---------------------------------------------
                
                preds_arr = np.zeros((x_val.shape[0], self.num_classes))
                targets_arr = np.zeros(y_val.shape[0])
                model.reset_new_repr(dataset_size=len(x_train))
                new_repr = None
                current_pos = 0
                model.train()
                for data, target in loader_train:
                    data = data.to(self.device)
                    target = target.to(self.device)
                    
                    preds = model(data)
                    loss = criterion(preds, target)
                    
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                
                if epoch+1 == epochs:
                    print('LGB training start')
                    new_repr = model.get_new_repr()
                    self.lgb_models.append(lgb_model)
                    self.lgb_models[fold].fit(new_repr, y_train)
                    print('LGB training end')
                
                model.reset_new_repr(dataset_size=len(x_val))
                
                with torch.no_grad():
                    model.eval()
                    for data, target in loader_val:
                        data = data.to(self.device)
                        target = target.to(self.device)

                        preds = model(data)
                        preds_arr[current_pos:current_pos+preds.shape[0]] = preds.cpu().numpy()
                        targets_arr[current_pos:current_pos+preds.shape[0]] = target.cpu().numpy()
                        current_pos += preds.shape[0]

                    if epoch+1 == epochs:
                        print('LGB validation start')
                        new_repr = model.get_new_repr()
                        lgb_preds_full[val_ind] = self.lgb_models[fold].predict_proba(new_repr)
                        nn_preds_full[val_ind] = preds_arr
                        print('LGB validation end')
                    
                print(f'Epoch: {epoch+1}')
            
            self.models.append(model)
            for i in range(len(self.noise_models)):
                self.noise_models[f'model{i+1}'].append(noise_model[f'model{i+1}'])
            
        return nn_preds_full, lgb_preds_full, noise_preds_full
    
    def predict(self, x):
        with torch.no_grad():
            x = np.array(x)
            preds = np.zeros((x.shape[0], self.num_classes))
            lgb_preds = np.zeros((x.shape[0], self.num_classes))
            noise_preds = np.zeros((len(self.noise_models), x.shape[0], self.num_classes))
            x = torch.tensor(x).float().to(self.device)
            for i in range(self.num_folds):
                for k in range(len(self.noise_models)):
                    self.noise_models[f'model{k+1}'][i].eval()
                    noise_preds[k] += self.noise_models[f'model{k+1}'][i](x).cpu().numpy()
                
                self.models[i].eval()
                self.models[i].reset_new_repr(dataset_size=x.shape[0])
                preds += self.models[i](x).cpu().numpy()

                new_repr = self.models[i].get_new_repr()
                lgb_preds += self.lgb_models[i].predict_proba(new_repr)

            for k in range(len(self.noise_models)):
                noise_preds[k] /= self.num_folds
            
            return preds / self.num_folds, lgb_preds / self.num_folds, noise_preds

In [11]:
class Pipeline:
    def __init__(self):
        self.new_repr_model = None
        self.dim_standard_model = None
        self.dim_reduced_model = None
        self.pca = None
        
        self.dim_standard_preds = None
        self.dim_reduced_preds = None
        self.dim_increased_preds = None
        self.nn_preds = None
        self.noise_preds = None
        
        self.num_classes = None
        self.target = None
        
        self.dim_reduced_weights = None
        self.dim_increased_weights = None
        self.dim_standard_weights = None
        self.nn_weights = None
        self.noise_weights = None
    
    def objective(self, trial):
        dim_reduced_trial = []
        dim_increased_trial = []
        dim_standard_trial = []
        nn_trial = []
        noise_trial = dict()
        for k in range(self.noise_preds.shape[0]):
            noise_trial[f'trial{k+1}'] = []
        for i in range(self.num_classes):
            dim_reduced_trial.append(trial.suggest_float(f'dim_reduced_weights{i+1}', 0.0, 1.0))
            dim_increased_trial.append(trial.suggest_float(f'dim_increased_weights{i+1}', 0.0, 1.0))
            dim_standard_trial.append(trial.suggest_float(f'dim_standard_weights{i+1}', 0.0, 1.0))
            nn_trial.append(trial.suggest_float(f'nn_weights{i+1}', 0.0, 1.0))
            for k in range(self.noise_preds.shape[0]):
                noise_trial[f'trial{k+1}'].append(trial.suggest_float(f'noise_weights{i+1}_{k+1}', 0.0, 1.0))
        
        all_weights_cols = []
        for i in range(self.num_classes):
            temp_weights = 0.0
            for k in range(self.noise_preds.shape[0]):
                temp_weights += noise_trial[f'trial{k+1}'][i]
            all_weights_cols.append(
                dim_reduced_trial[i] + dim_increased_trial[i] + dim_standard_trial[i] + nn_trial[i] + temp_weights
            )
        all_weights = sum(all_weights_cols)
        
        preds_probs = np.zeros((len(self.nn_preds), self.num_classes))
        
        self.dim_reduced_weights = []
        self.dim_increased_weights = []
        self.dim_standard_weights = []
        self.nn_weights = []
        self.noise_weights = dict()
        for k in range(self.noise_preds.shape[0]):
            self.noise_weights[f'weights{k+1}'] = []
        for i in range(self.num_classes):
            self.dim_reduced_weights.append(dim_reduced_trial[i]/all_weights_cols[i]/all_weights)
            self.dim_increased_weights.append(dim_increased_trial[i]/all_weights_cols[i]/all_weights)
            self.dim_standard_weights.append(dim_standard_trial[i]/all_weights_cols[i]/all_weights)
            self.nn_weights.append(nn_trial[i]/all_weights_cols[i]/all_weights)
            for k in range(self.noise_preds.shape[0]):
                self.noise_weights[f'weights{k+1}'].append(noise_trial[f'trial{k+1}'][i]/all_weights_cols[i]/all_weights)
            
            preds_probs[:, i] = self.dim_reduced_preds[:, i]*self.dim_reduced_weights[i] \
                              + self.dim_increased_preds[:, i]*self.dim_increased_weights[i] \
                              + self.dim_standard_preds[:, i]*self.dim_standard_weights[i] \
                              + self.nn_preds[:, i]*self.nn_weights[i]
            for k in range(self.noise_preds.shape[0]):
                preds_probs[:, i] += self.noise_preds[k, :, i]*self.noise_weights[f'weights{k+1}'][i]
        
        preds = np.argmax(preds_probs, axis=1)
        
        return accuracy_score(self.target, preds)
    
    def fit(self, data):
        
        self.target = data['target']
        
        columns = [f'col{i+1}' for i in range(180)]
        decomposed_data = pd.DataFrame()
        decomposed_data.index = data.index
        self.pca = PCA(n_components=180)
        decomposed_data.loc[:, columns] = self.pca.fit_transform(data.drop(['row_id', 'target'], axis=1))

        self.dim_reduced_model = LGBModel()
        self.dim_reduced_preds = self.dim_reduced_model.fit(decomposed_data, data['target'], is_plot=False, is_prob=True)
        
        self.dim_standard_model = LGBModel()
        self.dim_standard_preds = self.dim_standard_model.fit(data.drop(['target', 'row_id'], axis=1), data['target'], is_plot=False, is_prob=True)
        
        self.new_repr_model = NewReprModel()
        self.nn_preds, self.dim_increased_preds, self.noise_preds = self.new_repr_model.fit(data.drop(['row_id', 'target'], axis=1), data['target'])
        
        self.num_classes = self.nn_preds.shape[1]
        
        study = optuna.create_study(direction="maximize")
        study.optimize(self.objective, n_trials=2000)
        p = study.best_trial.params
        
        all_weights_cols = []
        for i in range(self.num_classes):
            temp_weights = 0.0
            for k in range(self.noise_preds.shape[0]):
                temp_weights += p[f'noise_weights{i+1}_{k+1}']
            all_weights_cols.append(
                p[f'dim_reduced_weights{i+1}'] + p[f'dim_increased_weights{i+1}'] \
                + p[f'dim_standard_weights{i+1}'] + p[f'nn_weights{i+1}'] + temp_weights
            )
        all_weights = sum(all_weights_cols)
        
        self.dim_reduced_weights = []
        self.dim_increased_weights = []
        self.dim_standard_weights = []
        self.nn_weights = []
        self.noise_weights = dict()
        for k in range(self.noise_preds.shape[0]):
            self.noise_weights[f'weights{k+1}'] = []
        for i in range(self.num_classes):
            self.dim_reduced_weights.append(p[f'dim_reduced_weights{i+1}']/all_weights_cols[i]/all_weights)
            self.dim_increased_weights.append(p[f'dim_increased_weights{i+1}']/all_weights_cols[i]/all_weights)
            self.dim_standard_weights.append(p[f'dim_standard_weights{i+1}']/all_weights_cols[i]/all_weights)
            self.nn_weights.append(p[f'nn_weights{i+1}']/all_weights_cols[i]/all_weights)
            for k in range(self.noise_preds.shape[0]):
                self.noise_weights[f'weights{k+1}'].append(p[f'noise_weights{i+1}_{k+1}']/all_weights_cols[i]/all_weights)
    
    def predict(self, x):
        dim_reduced_preds = self.dim_reduced_model.predict(self.pca.transform(x))
        dim_standard_preds = self.dim_standard_model.predict(x)
        nn_preds, dim_increased_preds, noise_preds = self.new_repr_model.predict(x)
        
        preds_probs = np.zeros((len(nn_preds), self.num_classes))
        
        for i in range(self.num_classes):
            preds_probs[:, i] = dim_reduced_preds[:, i]*self.dim_reduced_weights[i] \
                              + dim_increased_preds[:, i]*self.dim_increased_weights[i] \
                              + dim_standard_preds[:, i]*self.dim_standard_weights[i] \
                              + nn_preds[:, i]*self.nn_weights[i]
            for k in range(self.noise_preds.shape[0]):
                preds_probs[:, i] += noise_preds[k, :, i]*self.noise_weights[f'weights{k+1}'][i]
        
        preds = np.argmax(preds_probs, axis=1)
        
        return preds

In [ ]:
pipe = Pipeline()
pipe.fit(df_train)

In [15]:
submission_df = df_test[['row_id']]
submission_df['target'] = pipe.predict(df_test.drop('row_id', axis=1))
submission_df['target'] = submission_df['target'].replace(val2key)
submission_df.to_csv('submission.csv', index=False)

"submission_df = df_test[['row_id']]\nsubmission_df['target'] = pipe.predict(df_test.drop('row_id', axis=1))\nsubmission_df['target'] = submission_df['target'].replace(val2key)\nsubmission_df.to_csv('submission.csv', index=False)"